# STUDIO PRELIMINARE

#### Maraziti Matteo, Tocci Federico e Scordino Giacomo

[Link Repository](https://github.com/FTocci/TGW2D)


Le istruzioni presentate di seguito sono state utilizzate per la realizzazione dei primi test. L'obiettivo è stato quello di verificare il corretto funzionamento dell'algoritmo e di individuare i moduli di interesse.
Con l'utilizzo di ViewerGL è stato possibile visualizzare graficamente il risultato delle avvenute elaborazioni.

In [1]:
using ViewerGL
GL = ViewerGL
using LinearAlgebraicRepresentation
Lar = LinearAlgebraicRepresentation
using SparseArrays

include("../src/arrangement/planar_arrangement.jl") 

V = [2 2; 1.5 2; 3 3.5; 1 3; 5 3; 1 2; 5 2]
EV = SparseArrays.sparse(Array{Int8, 2}([
               [1 1 0 0 0 0 0] #1->1,2
               [0 1 1 0 0 0 0] #2->2,3
               [1 0 1 0 0 0 0] #3->1,3
               [0 0 0 1 1 0 0] #4->4,5
               [0 0 0 0 0 1 1] #5->6,7
           ]))
sigma=spzeros(Int8, 0)
return_edge_map=false
multiproc=false
W = convert(Lar.Points,V')

V, copEV, copFE=planar_arrangement(V,EV,sigma,return_edge_map,multiproc)

EV = Lar.cop2lar(copEV)
FE = Lar.cop2lar(copFE)
FV=EV
FV = convert(Lar.Cells,FV)
VV = [[k] for k=1:size(V,2)]
model = (W, [VV,EV,FV])::Lar.LARmodel;
meshes = GL.numbering(1.5)(model, GL.COLORS[1], 0.1);
GL.VIEW(meshes);

### Analisi Tempistiche

Dopo aver constatato il corretto funzionamento del codice principale bisogna effettuare analisi in termini di tempistiche di esecuzione dell'algoritmo.

La funzione planar_arrangement è costituita da un codice molto minimale, in quanto la parte più importante dell'esecuzione avviene nelle funzioni esterne che sono chiamate all'interno di essa. 

In [22]:
function planar_arrangement(V::Lar.Points, copEV::Lar.ChainOp, sigma::Lar.Chain=spzeros(Int8, 0),
    return_edge_map::Bool=false,
    multiproc::Bool=false)
#planar_arrangement_1
V,copEV,sigma,edge_map=planar_arrangement_1(V,copEV,sigma,return_edge_map,multiproc)
# cleandecomposition
if sigma.n > 0
    V,copEV=cleandecomposition(V, copEV, sigma, edge_map)
end
bicon_comps = Lar.Arrangement.biconnected_components(copEV)
if isempty(bicon_comps)
    println("No biconnected components found.")
    if (return_edge_map)
        return (nothing, nothing, nothing, nothing)
    else
        return (nothing, nothing, nothing)
    end
end
#Planar_arrangement_2
V,copEV,FE=Lar.Arrangement.planar_arrangement_2(V,copEV,bicon_comps,edge_map,sigma)
if (return_edge_map)
     return V, copEV, FE, edge_map
else
     return V, copEV, FE
end
end

planar_arrangement (generic function with 4 methods)

Le funzioni chiamate internamente alla funzione planar_arrangement sono:
1. planar_arrangement_1
2. cleandecomposition
3. biconnected_components
4. planar_arrangement_2

### Prototipi funzioni da ottimizzare

In [ ]:
function planar_arrangement_1( V, copEV,
    sigma::Lar.Chain=spzeros(Int8, 0),
    return_edge_map::Bool=false,
    multiproc::Bool=false)
# data structures initialization
edgenum = size(copEV, 1)
edge_map = Array{Array{Int, 1}, 1}(undef,edgenum)
rV = Lar.Points(zeros(0, 2))
rEV = SparseArrays.spzeros(Int8, 0, 0)
finalcells_num = 0

# spaceindex computation
model = (convert(Lar.Points,V'),Lar.cop2lar(copEV))
bigPI = Lar.spaceindex(model::Lar.LAR)

# multiprocessing of edge fragmentation
if (multiproc == true)
    in_chan = Distributed.RemoteChannel(()->Channel{Int64}(0))
    out_chan = Distributed.RemoteChannel(()->Channel{Tuple}(0))
    ordered_dict = SortedDict{Int64,Tuple}()
    @async begin
        for i in 1:edgenum
            put!(in_chan,i)
        end
        for p in distributed.workers()
            put!(in_chan,-1)
        end
    end
    for p in distributed.workers()
        @async Base.remote_do(frag_edge_channel, p, in_chan, out_chan, V, copEV, bigPI)
    end
    for i in 1:edgenum
        frag_done_job = take!(out_chan)
        ordered_dict[frag_done_job[1]] = frag_done_job[2]
    end
    for (dkey, dval) in ordered_dict
        i = dkey
        v, ev = dval
        newedges_nums = map(x->x+finalcells_num, collect(1:size(ev, 1)))
        edge_map[i] = newedges_nums
        finalcells_num += size(ev, 1)
        rV, rEV = Lar.skel_merge(rV, rEV, v, ev)
    end
else
    # sequential (iterative) processing of edge fragmentation
    for i in 1:edgenum
        v, ev = frag_edge(V, copEV, i, bigPI)
        newedges_nums = map(x->x+finalcells_num, collect(1:size(ev, 1)))
        edge_map[i] = newedges_nums
        finalcells_num += size(ev, 1)
        rV = convert(Lar.Points, rV)
        rV, rEV = Lar.skel_merge(rV, rEV, v, ev)
    end
end
# merging of close vertices and edges (2D congruence)
V, copEV = rV, rEV
V, copEV = merge_vertices!(V, copEV, edge_map)
return V,copEV,sigma,edge_map
end

In [ ]:
function planar_arrangement_2(V, copEV,bicon_comps, edge_map,
    sigma::Lar.Chain=spzeros(Int8, 0))

edges = sort(union(bicon_comps...))
todel = sort(setdiff(collect(1:size(copEV,1)), edges))

for i in reverse(todel)
    for row in edge_map

        filter!(x->x!=i, row)

        for j in 1:length(row)
            if row[j] > i
                row[j] -= 1
            end
        end
    end
end
bicon_comps = Lar.Arrangement.biconnected_components(copEV)
# component graph
n, containment_graph, V, EVs, boundaries, shells, shell_bboxes=Lar.Arrangement.componentgraph(V,copEV,bicon_comps)
copEV, FE = Lar.Arrangement.cell_merging(
       n, containment_graph, V, EVs, boundaries, shells, shell_bboxes)
return V,copEV,FE
end

In [ ]:
function cleandecomposition(V, copEV, sigma, edge_map)
    # Deletes edges outside sigma area
    todel = []
    new_edges = []
    map(i->new_edges=union(new_edges, edge_map[i]), sigma.nzind)
    ev = copEV[new_edges, :]
    for e in 1:copEV.m
        if !(e in new_edges)

            vidxs = copEV[e, :].nzind
            v1, v2 = map(i->V[vidxs[i], :], [1,2])
            centroid = .5*(v1 + v2)

            if ! Lar.point_in_face(centroid, V, ev)
                push!(todel, e)
            end
        end
    end

    for i in reverse(todel)
        for row in edge_map

            filter!(x->x!=i, row)

            for j in 1:length(row)
                if row[j] > i
                    row[j] -= 1
                end
            end
        end
    end

    V, copEV = Lar.delete_edges(todel, V, copEV)
	return V,copEV
end

In [ ]:
function biconnected_components(EV::Lar.ChainOp)

    ps = Array{Tuple{Int, Int, Int}, 1}()
    es = Array{Tuple{Int, Int}, 1}()
    todel = Array{Int, 1}()
    visited = Array{Int, 1}()
    bicon_comps = Array{Array{Int, 1}, 1}()
    hivtx = 1

    function an_edge(point) # TODO: fix bug
        # error? : BoundsError: attempt to access 0×0 SparseMatrix ...
        edges = setdiff(EV[:, point].nzind, todel)
        if length(edges) == 0
            edges = [false]
        end
        edges[1]
    end

    function get_head(edge, tail)
        setdiff(EV[edge, :].nzind, [tail])[1]
    end

    function v_to_vi(v)
        i = findfirst(t->t[1]==v, ps)
        # seems findfirst changed from 0 to Nothing
        if typeof(i) == Nothing
            return false
        elseif i == 0
            return false
        else
            return ps[i][2]
        end
    end

    push!(ps, (1,1,1))
    push!(visited, 1)
    exit = false
    while !exit
        edge = an_edge(ps[end][1])
        if edge != false
            tail = ps[end][2]
            head = get_head(edge, ps[end][1])
            hi = v_to_vi(head)
            if hi == false
                hivtx += 1
                push!(ps, (head, hivtx, ps[end][2]))
                push!(visited, head)
            else
                if hi < ps[end][3]
                    ps[end] = (ps[end][1], ps[end][2], hi)
                end
            end
            push!(es, (edge, tail))
            push!(todel, edge)
        else
            if length(ps) == 1
                found = false
                pop!(ps)
                for i in 1:size(EV,2)
                    if !(i in visited)
                        hivtx = 1
                        push!(ps, (i, hivtx, 1))
                        push!(visited, i)
                        found = true
                        break
                    end
                end
                if !found
                    exit = true
                end

            else
                if ps[end][3] == ps[end-1][2]
                    edges = Array{Int, 1}()
                    while true
                        edge, tail = pop!(es)
                        push!(edges, edge)
                        if tail == ps[end][3]
                            if length(edges) > 1
                                push!(bicon_comps, edges)
                            end
                            break
                        end
                    end

                else
                    if ps[end-1][3] > ps[end][3]
                        ps[end-1] = (ps[end-1][1], ps[end-1][2], ps[end][3])
                    end
                end
                pop!(ps)
            end
        end
    end
    bicon_comps = sort(bicon_comps, lt=(x,y)->length(x)>length(y))
    return bicon_comps
end

È possibile migliorare le tempistiche operando internamente alle presenti funzioni.

### Conclusioni

All'interno di ciascuna di queste funzioni vi sono ulteriori chiamate a funzioni esterne, in queste ultime si identifica come fattor comune che innalza notevolmente i tempi di esecuzione la vasta presenza di cicli for, i quali dovranno essere eseguiti per ciascun elemento da elaborare. 
Come ovviare questo problema verrà discusso all'interno del [Notebook finale](NotebookFinale.ipynb).